In [185]:
import pandas as pd
data_dir = "../data/"
output_dir = "../output/"

In [241]:
thresh = 0.00001 # 1e-5
species = pd.read_csv("../data/taxonomy.csv", index_col = 0).T
pathways = pd.read_csv("../data/pathways.csv", index_col = 0).T
X = pd.concat([species, pathways], axis = 1)
# X = species
X_old = X.copy()
X = X > thresh
X

,s__Abiotrophia_defectiva,s__Acidaminococcus_fermentans,s__Acidaminococcus_intestini,s__Actinomyces_graevenitzii,s__Actinomyces_odontolyticus,s__Actinomyces_oris,s__Actinomyces_turicensis,s__Actinomyces_viscosus,s__Adlercreutzia_equolifaciens,s__Aggregatibacter_segnis,...,TRNA-CHARGING-PWY: tRNA charging,TRPSYN-PWY: L-tryptophan biosynthesis,TYRFUMCAT-PWY: L-tyrosine degradation I,UBISYN-PWY: superpathway of ubiquinol-8 biosynthesis (prokaryotic),UDPNACETYLGALSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis II,UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,URDEGR-PWY: superpathway of allantoin degradation in plants,URSIN-PWY: ureide biosynthesis,VALDEG-PWY: L-valine degradation I,VALSYN-PWY: L-valine biosynthesis
SAMD00036192,False,False,False,True,True,False,False,False,False,False,...,True,True,False,True,False,True,False,False,False,True
SAMD00036193,False,False,False,True,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036194,False,False,False,False,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036197,False,False,False,True,True,False,False,True,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036204,False,False,False,True,True,True,False,False,True,False,...,True,True,False,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN07534852,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,False,True,True,False,False,True
SAMN07534870,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,False,True,True,False,False,True
SAMN07534977,False,False,False,False,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMN07534978,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,False,True,True,False,False,True


In [242]:
y = pd.read_csv("../data/isHealthy.csv", index_col=0).T
y

,isHealthy
SAMD00036192,True
SAMD00036193,True
SAMD00036194,False
SAMD00036197,True
SAMD00036204,True
...,...
SAMN07534852,True
SAMN07534870,True
SAMN07534977,True
SAMN07534978,True


In [292]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [293]:
pivot = pd.concat([X_train[y_train['isHealthy']].mean(), X_train[~y_train['isHealthy']].mean()], axis = 1)
pivot.columns = ['healthy', 'not']
pivot['diff'] = pivot['healthy'] - pivot['not']
pivot['foldh'] = (pivot['healthy'] + 0.0001) / (pivot['not'] + 0.0001)
pivot['foldn'] = (pivot['not'] + 0.0001) / (pivot['healthy'] + 0.0001)

In [294]:
pivot['weight'] = 0
pivot

,healthy,not,diff,foldh,foldn,weight
s__Abiotrophia_defectiva,0.007179,0.047927,-0.040748,0.151560,6.598036,0
s__Acidaminococcus_fermentans,0.037162,0.077073,-0.039910,0.482842,2.071070,0
s__Acidaminococcus_intestini,0.080659,0.117228,-0.036569,0.688316,1.452820,0
s__Actinomyces_graevenitzii,0.044341,0.121114,-0.076773,0.366634,2.727513,0
s__Actinomyces_odontolyticus,0.090794,0.150259,-0.059465,0.604512,1.654226,0
...,...,...,...,...,...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.990709,0.989637,0.001072,1.001083,0.998918,0
URDEGR-PWY: superpathway of allantoin degradation in plants,0.296030,0.438472,-0.142441,0.675216,1.481008,0
URSIN-PWY: ureide biosynthesis,0.002534,0.004534,-0.002000,0.568400,1.759324,0
VALDEG-PWY: L-valine degradation I,0.005912,0.003238,0.002674,1.800943,0.555265,0


In [295]:
import numpy as np
pivot.loc[pivot['diff'] > 0, 'weight'] = np.log(pivot[pivot['diff'] > 0]['foldh'])
pivot

,healthy,not,diff,foldh,foldn,weight
s__Abiotrophia_defectiva,0.007179,0.047927,-0.040748,0.151560,6.598036,0.000000
s__Acidaminococcus_fermentans,0.037162,0.077073,-0.039910,0.482842,2.071070,0.000000
s__Acidaminococcus_intestini,0.080659,0.117228,-0.036569,0.688316,1.452820,0.000000
s__Actinomyces_graevenitzii,0.044341,0.121114,-0.076773,0.366634,2.727513,0.000000
s__Actinomyces_odontolyticus,0.090794,0.150259,-0.059465,0.604512,1.654226,0.000000
...,...,...,...,...,...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.990709,0.989637,0.001072,1.001083,0.998918,0.001083
URDEGR-PWY: superpathway of allantoin degradation in plants,0.296030,0.438472,-0.142441,0.675216,1.481008,0.000000
URSIN-PWY: ureide biosynthesis,0.002534,0.004534,-0.002000,0.568400,1.759324,0.000000
VALDEG-PWY: L-valine degradation I,0.005912,0.003238,0.002674,1.800943,0.555265,0.588310


In [296]:
pivot.loc[pivot['diff'] < 0, 'weight'] = np.log(pivot[pivot['diff'] < 0]['foldn'])
pivot.head()

,healthy,not,diff,foldh,foldn,weight
s__Abiotrophia_defectiva,0.007179,0.047927,-0.040748,0.151560,6.598036,1.886772
s__Acidaminococcus_fermentans,0.037162,0.077073,-0.039910,0.482842,2.071070,0.728065
s__Acidaminococcus_intestini,0.080659,0.117228,-0.036569,0.688316,1.452820,0.373507
s__Actinomyces_graevenitzii,0.044341,0.121114,-0.076773,0.366634,2.727513,1.003390
s__Actinomyces_odontolyticus,0.090794,0.150259,-0.059465,0.604512,1.654226,0.503333


In [297]:
pivot['weight2'] = abs(pivot['weight'] * pivot['diff'])
pivot

,healthy,not,diff,foldh,foldn,weight,weight2
s__Abiotrophia_defectiva,0.007179,0.047927,-0.040748,0.151560,6.598036,1.886772,0.076883
s__Acidaminococcus_fermentans,0.037162,0.077073,-0.039910,0.482842,2.071070,0.728065,0.029057
s__Acidaminococcus_intestini,0.080659,0.117228,-0.036569,0.688316,1.452820,0.373507,0.013659
s__Actinomyces_graevenitzii,0.044341,0.121114,-0.076773,0.366634,2.727513,1.003390,0.077033
s__Actinomyces_odontolyticus,0.090794,0.150259,-0.059465,0.604512,1.654226,0.503333,0.029931
...,...,...,...,...,...,...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.990709,0.989637,0.001072,1.001083,0.998918,0.001083,0.000001
URDEGR-PWY: superpathway of allantoin degradation in plants,0.296030,0.438472,-0.142441,0.675216,1.481008,0.392723,0.055940
URSIN-PWY: ureide biosynthesis,0.002534,0.004534,-0.002000,0.568400,1.759324,0.564930,0.001130
VALDEG-PWY: L-valine degradation I,0.005912,0.003238,0.002674,1.800943,0.555265,0.588310,0.001573


In [298]:
X_good = X_train[pivot[pivot['diff'] > 0].nlargest(214,
    "weight2").index]
X_good_test = X_test[pivot[pivot['diff'] > 0].nlargest(214,
    "weight2").index]
X_good

,s__Bifidobacterium_angulatum,s__Bifidobacterium_catenulatum,s__Sutterella_wadsworthensis,s__Bifidobacterium_adolescentis,s__Lachnospiraceae_bacterium_8_1_57FAA,s__Bacteroidales_bacterium_ph8,s__Alistipes_senegalensis,PWY-5871: ubiquinol-9 biosynthesis (eukaryotic),PWY-5873: ubiquinol-7 biosynthesis (eukaryotic),s__Lactobacillus_ruminis,...,DENOVOPURINE2-PWY: superpathway of purine nucleotides de novo biosynthesis II,PWY-2941: L-lysine biosynthesis II,PWY-7539: 6-hydroxymethyl-dihydropterin diphosphate biosynthesis III (Chlamydia),PWY-6527: stachyose degradation,PWY-6317: galactose degradation I (Leloir pathway),PWY-841: superpathway of purine nucleotides de novo biosynthesis I,PWY-5177: glutaryl-CoA degradation,PWY-7187: pyrimidine deoxyribonucleotides de novo biosynthesis II,PWY-5100: pyruvate fermentation to acetate and lactate II,PWY0-845: superpathway of pyridoxal 5'-phosphate biosynthesis and salvage
SAMEA3879600,False,False,False,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,True,True,True
SAMN00715181,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
SAMN00142634,False,False,True,False,False,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True
SAMEA3664664,False,True,False,True,False,True,True,False,False,True,...,True,True,True,True,True,True,True,True,True,True
SAMN00142107,False,False,True,True,False,True,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN00792087,False,False,True,False,False,True,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
SAMEA1972159,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
SAMN00070035,False,False,True,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
SAMN06110010,False,True,False,False,False,True,True,False,False,False,...,True,True,True,True,True,True,True,True,True,True


In [299]:
X_bad = X_train[pivot[pivot['diff'] < 0].nlargest(214,
    "weight2").index]
X_bad_test = X_test[pivot[pivot['diff'] < 0].nlargest(214,
    "weight2").index]
X_bad

,s__Solobacterium_moorei,s__Granulicatella_adiacens,s__Clostridium_ramosum,s__Clostridium_bolteae,s__Peptostreptococcus_stomatis,s__Clostridium_symbiosum,s__Clostridiales_bacterium_1_7_47FAA,s__Clostridium_hathewayi,s__candidate_division_TM7_single_cell_isolate_TM7c,s__Streptococcus_anginosus,...,s__Clostridium_celatum,"PWY-5791: 1,4-dihydroxy-2-naphthoate biosynthesis II (plants)","PWY-5837: 1,4-dihydroxy-2-naphthoate biosynthesis I",s__Streptococcus_infantarius,s__Pyramidobacter_piscolens,s__Clostridiaceae_bacterium_JC118,PWY-5863: superpathway of phylloquinol biosynthesis,PWY-5675: nitrate reduction V (assimilatory),PWY-4321: L-glutamate degradation IV,s__Clostridium_hylemonae
SAMEA3879600,False,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,True,False,False,False
SAMN00715181,False,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,False,False,False,False
SAMN00142634,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SAMEA3664664,True,False,False,True,True,False,True,False,False,False,...,False,True,True,False,False,True,True,False,False,False
SAMN00142107,False,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN00792087,True,True,False,True,False,False,False,False,False,False,...,False,True,True,False,False,False,True,False,False,False
SAMEA1972159,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
SAMN00070035,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SAMN06110010,False,False,False,False,False,False,False,True,False,False,...,False,True,True,False,False,False,True,False,False,False


In [300]:
theta_good = pivot[pivot['diff'] > 0]['weight2'].sort_values(
ascending=False)
print(theta_good.shape)
theta_good

(210,)


s__Bifidobacterium_angulatum                                                 2.134920e-01
s__Bifidobacterium_catenulatum                                               1.412840e-01
s__Sutterella_wadsworthensis                                                 1.262045e-01
s__Bifidobacterium_adolescentis                                              8.854507e-02
s__Lachnospiraceae_bacterium_8_1_57FAA                                       8.481894e-02
                                                                                 ...     
PWY-841: superpathway of purine nucleotides de novo biosynthesis I           2.301383e-07
PWY-5177: glutaryl-CoA degradation                                           7.991072e-08
PWY-7187: pyrimidine deoxyribonucleotides de novo biosynthesis II            5.081423e-08
PWY-5100: pyruvate fermentation to acetate and lactate II                    5.081423e-08
PWY0-845: superpathway of pyridoxal 5'-phosphate biosynthesis and salvage    2.935071e-08
Name: weig

In [301]:
theta_bad = pivot[pivot['diff'] < 0]['weight2'].nlargest(214)
print(theta_bad.shape)
theta_bad

(214,)


s__Solobacterium_moorei                                0.368499
s__Granulicatella_adiacens                             0.329314
s__Clostridium_ramosum                                 0.240393
s__Clostridium_bolteae                                 0.238182
s__Peptostreptococcus_stomatis                         0.223071
                                                         ...   
s__Clostridiaceae_bacterium_JC118                      0.033169
PWY-5863: superpathway of phylloquinol biosynthesis    0.031932
PWY-5675: nitrate reduction V (assimilatory)           0.031706
PWY-4321: L-glutamate degradation IV                   0.031667
s__Clostridium_hylemonae                               0.030893
Name: weight2, Length: 214, dtype: float64

In [302]:
y_numpy = y_train.to_numpy().flatten()
y_numpy_test = y_test.to_numpy().flatten()
y_numpy

array([ True,  True,  True, ...,  True,  True, False])

In [303]:
healthyIdx = np.where(y_train)[0]
unhealthyIdx = np.where(~y_train)[0]
healthyIdx_test = np.where(y_test)[0]
unhealthyIdx_test = np.where(~y_test)[0]
unhealthyIdx

array([   9,   11,   12, ..., 3907, 3908, 3911])

In [304]:
theta_good.sum(), theta_bad.sum()

(2.3678845207178507, 18.644071580865955)

In [305]:
X_good2 = X_good.to_numpy().astype(float)
X_bad2 = X_bad.to_numpy().astype(float)
theta_good2 = theta_good.to_numpy()
theta_bad2 = theta_bad.to_numpy()
X_good2_test = X_good_test.to_numpy().astype(float)
X_bad2_test = X_bad_test.to_numpy().astype(float)

In [306]:
theta_goods = np.vstack([theta_good2] * theta_good2.shape[0])
for i in range(theta_good2.shape[0]):
    theta_goods[i, i:] = 0
theta_goods[10]

array([0.21349199, 0.14128405, 0.12620454, 0.08854507, 0.08481894,
       0.08278796, 0.07982981, 0.07020478, 0.07020478, 0.06712703,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [307]:
theta_bads = np.vstack([theta_bad2] * theta_bad2.shape[0])
for i in range(theta_bad2.shape[0]):
    theta_bads[i, i:] = 0
theta_bads[20, :]

array([0.36849887, 0.32931399, 0.24039344, 0.23818227, 0.22307111,
       0.21338289, 0.21225053, 0.21179569, 0.20985811, 0.20149964,
       0.20047684, 0.19421738, 0.1931189 , 0.19107523, 0.18773107,
       0.18526934, 0.17871955, 0.1683065 , 0.16813999, 0.15946743,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [308]:
# 125, 203
from numba import njit
@njit(nogil=True)
def getChi(X_good, X_bad,
           theta_goods, theta_bads, healthyIdx, unhealthyIdx, num_good, num_bad):
    good_term = (X_good @ theta_goods[num_good, :])
    bad_term = (X_bad @ theta_bads[num_bad, :])
    y_hat = good_term - bad_term
    y_hat = y_hat.reshape(len(y_hat), 1)
    p_h = (y_hat[healthyIdx] > 0).sum() / len(healthyIdx)
    p_n = (y_hat[unhealthyIdx] < 0).sum() / len(unhealthyIdx)
    return 0.5 * (p_h + p_n)

In [314]:
%%time
getChi(X_good2, X_bad2,
        theta_goods, theta_bads,
        healthyIdx, unhealthyIdx, 
      56, 29)

CPU times: user 6.25 ms, sys: 855 µs, total: 7.11 ms
Wall time: 834 µs


0.733111390211455

In [310]:
# from numba import prange
# @njit(nogil=True, parallel=True)
def run():
    scores = [0.0]
    for num_good in range(theta_good2.shape[0]):
        for num_bad in range(theta_bad2.shape[0]):
            chi = getChi(X_good2, X_bad2, 
                theta_goods, theta_bads,
                healthyIdx, unhealthyIdx, 
                num_good, num_bad)
            scores.append(chi)
    return scores

In [311]:
%%time
scores = run()
max(scores)

CPU times: user 1min 34s, sys: 640 ms, total: 1min 34s
Wall time: 11.9 s


0.733111390211455

In [313]:
scores.index(max(scores)) % theta_bad2.shape[0]

30

In [315]:
getChi(X_good2_test, X_bad2_test,
        theta_goods, theta_bads,
        healthyIdx_test, unhealthyIdx_test, 
      56, 29)

0.7470841898292966

In [319]:
X_features = pd.concat([X_good.iloc[:, :56], X_bad.iloc[:, :29]], axis=1)
X_features_test = pd.concat([X_good_test.iloc[:, :56], X_bad_test.iloc[:, :29]], axis=1)
X_features

,s__Bifidobacterium_angulatum,s__Bifidobacterium_catenulatum,s__Sutterella_wadsworthensis,s__Bifidobacterium_adolescentis,s__Lachnospiraceae_bacterium_8_1_57FAA,s__Bacteroidales_bacterium_ph8,s__Alistipes_senegalensis,PWY-5871: ubiquinol-9 biosynthesis (eukaryotic),PWY-5873: ubiquinol-7 biosynthesis (eukaryotic),s__Lactobacillus_ruminis,...,PWY66-389: phytol degradation,s__Fusobacterium_varium,s__Lachnospiraceae_bacterium_1_4_56FAA,s__Streptococcus_sanguinis,s__Streptococcus_cristatus,s__Streptococcus_tigurinus,s__Clostridium_nexile,s__Streptococcus_infantis,METHGLYUT-PWY: superpathway of methylglyoxal degradation,s__Lactobacillus_salivarius
SAMEA3879600,False,False,False,True,True,True,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
SAMN00715181,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
SAMN00142634,False,False,True,False,False,True,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
SAMEA3664664,False,True,False,True,False,True,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
SAMN00142107,False,False,True,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN00792087,False,False,True,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,True,True,False
SAMEA1972159,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
SAMN00070035,False,False,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SAMN06110010,False,True,False,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [320]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=0.04, fit_intercept=False, max_iter=500, random_state=42).fit(
    X_features, y_train)
clf.score(X_features_test, y_test)

/Users/daniel/opt/anaconda3/envs/R/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.774712643678161